Working Code for Loan Prediction Problem

In [66]:
import pandas as pd
import numpy as np 
%matplotlib inline
import seaborn as sns

from fastai.imports import *
from fastai.structured import *
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import math
from scipy import stats 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier

In [ ]:
train_data=pd.read_csv("train.csv")

testing_data=pd.read_csv("test.csv")

train_copy=train_data.copy()

train_data.columns

# def fill_variables(column,fillingval,inplaceornot):
#     print(column.fillna(fillingval,inplace=inplaceornot))
#     return column.fillna(fillingval,inplace=inplaceornot)

# Custom Features (Has Dependents,Credit_History,Semiurban or not etc)

conditions = [train_data['Loan_Status'] == 'Y', train_data['Loan_Status'] == 'N']
values = [1.0, 0.0]
train_data['Credit_History'] = np.where(train_data['Credit_History'].isnull(),
                              np.select(conditions, values),
                              train_data['Credit_History'])

# testing_data['Credit_History'] = np.where(testing_data['Credit_History'].isnull(),
#                               np.select(conditions4, values),
#                               train_data['Credit_History'])

conditions2=[
    (train_data['Loan_Status']=='Y') & (train_data['Dependents']=='0'),
    (train_data['Loan_Status']=='N') & (train_data['Dependents']=='0'),
    (train_data['Loan_Status']=='Y') & (train_data['Dependents']=='1'),
    (train_data['Loan_Status']=='N') & (train_data['Dependents']=='1'),
    (train_data['Loan_Status']=='Y') & (train_data['Dependents']=='2'),
    (train_data['Loan_Status']=='N') & (train_data['Dependents']=='2'),
    (train_data['Loan_Status']=='Y') & (train_data['Dependents']=='3'),
    (train_data['Loan_Status']=='N') & (train_data['Dependents']=='3'),
]

condition3=[(train_data['Loan_Status']=='Y') & (train_data['Property_Area']=='Rural'),
    (train_data['Loan_Status']=='N') & (train_data['Property_Area']=='Rural'),
    (train_data['Loan_Status']=='Y') & (train_data['Property_Area']=='Urban'),
    (train_data['Loan_Status']=='N') & (train_data['Property_Area']=='Urban'),
    (train_data['Loan_Status']=='Y') & (train_data['Property_Area']=='Semiurban'),
    (train_data['Loan_Status']=='N') & (train_data['Property_Area']=='Semiurban'),]

train_data['Dependents'].fillna(train_data['Dependents'].mode()[0], inplace=True)
train_data['Dependents'].replace('3+',3,inplace=True)
# print(train_data['Dependents'])
values2=[1,0,0,0,0,0,0,0]
values3=[0,0,0,0,1,0]

train_data['HasDependent']=np.select(conditions2,values2)
testing_data['HasDependent']=[1 if x=='0' else 0 for x in testing_data['Dependents']]
# train_data['HasDependent']
train_data['IsSemiUrban']=np.select(condition3,values3)
testing_data['IsSemiUrban']=[1 if x=='Semiurban' else 0 for x in testing_data['Property_Area']]
# print(train_data['IsSemiUrban'])
# train_data['HasDependent']=[train_data['Dependents']=='0' & train_data['Loan_Status']=='Y']
testing_data['Dependents'].fillna(train_data['Dependents'].mode()[0], inplace=True)
testing_data['Dependents'].replace('3+',3,inplace=True)

# testing_data['HasDependent']=

# train_data['HasDependent']

In [68]:
train_data['Gender']=train_data['Gender'].fillna(train_data['Gender'].mode()[0], inplace=True)
train_data['Married'].fillna(train_data['Married'].mode()[0], inplace=True)
# train_data['Dependents'].fillna(train_data['Dependents'].mode()[0], inplace=True)
train_data['Self_Employed'].fillna(train_data['Self_Employed'].mode()[0], inplace=True)
# train_data['Credit_History'].fillna(train_data['Credit_History'].mode()[0], inplace=True)
train_data['Loan_Amount_Term'].fillna(train_data['Loan_Amount_Term'].mode()[0], inplace=True)
train_data['LoanAmount'].fillna(train_data['LoanAmount'].median(), inplace=True)
train_data['LoanAmount']=np.log(train_data['LoanAmount'])
train_data['ApplicantIncome'].fillna(train_data['ApplicantIncome'].median(),inplace=True)
train_data['CoapplicantIncome'].fillna(train_data['CoapplicantIncome'].median(),inplace=True)
train_data['Total_Income']=np.log(train_data['ApplicantIncome']+train_data['CoapplicantIncome'])
# train_data['Dependents'].replace('0','N',inplace=True)
# train_data['Dependents'].replace('1','Y1',inplace=True)
# train_data['Dependents'].replace('2','Y2',inplace=True)
# train_data['Dependents'].replace('3+','Y3',inplace=True)
# train_data['Married+Dependents']=train_data['Married']+train_data['Dependents']

testing_data['Gender'].fillna(testing_data['Gender'].mode()[0], inplace=True)
testing_data['Married'].fillna(testing_data['Married'].mode()[0], inplace=True)
# testing_data['Dependents'].fillna(testing_data['Dependents'].mode()[0], inplace=True)
testing_data['Self_Employed'].fillna(testing_data['Self_Employed'].mode()[0], inplace=True)
testing_data['Credit_History'].fillna(testing_data['Credit_History'].mode()[0], inplace=True)
testing_data['Loan_Amount_Term'].fillna(testing_data['Loan_Amount_Term'].mode()[0], inplace=True)
testing_data['LoanAmount'].fillna(testing_data['LoanAmount'].median(), inplace=True)
testing_data['LoanAmount']=np.log(testing_data['LoanAmount'])
testing_data['ApplicantIncome'].fillna(testing_data['ApplicantIncome'].median(),inplace=True)
testing_data['CoapplicantIncome'].fillna(testing_data['CoapplicantIncome'].median(),inplace=True)
testing_data['Total_Income']=np.log(testing_data['ApplicantIncome']+testing_data['CoapplicantIncome'])
# testing_data['Dependents'].replace('0','N',inplace=True)
# testing_data['Dependents'].replace('1','Y1',inplace=True)
# testing_data['Dependents'].replace('2','Y2',inplace=True)
# testing_data['Dependents'].replace('3+','Y3',inplace=True)
# testing_data['Married+Dependents']=testing_data['Married']+testing_data['Dependents']

train_data['EMI']=train_data['LoanAmount']/train_data['Loan_Amount_Term']
testing_data['EMI']=testing_data['LoanAmount']/testing_data['Loan_Amount_Term']

In [69]:
train_data.drop('Loan_ID',inplace=True,axis=1)
# train_data['Dependents'].replace('3+',3,inplace=True)
train_data.drop('Married',inplace=True,axis=1)
train_data.drop('ApplicantIncome',inplace=True,axis=1)
train_data.drop('CoapplicantIncome',inplace=True,axis=1)
# train_data.drop('Dependents',inplace=True,axis=1)
train_data.drop('Loan_Amount_Term',inplace=True,axis=1)

original_Ids=testing_data['Loan_ID']

testing_data.drop('Loan_ID',inplace=True,axis=1)
# testing_data['Dependents'].replace('3+',3,inplace=True)
testing_data.drop('Married',inplace=True,axis=1)
testing_data.drop('ApplicantIncome',inplace=True,axis=1)
testing_data.drop('CoapplicantIncome',inplace=True,axis=1)
# testing_data.drop('Dependents',inplace=True,axis=1)
testing_data.drop('Loan_Amount_Term',inplace=True,axis=1)
# testing_data.replace('NoN',0,inplace=True)

In [70]:
# train_data=pd.get_dummies(train_data)
# testing_data=pd.get_dummies(testing_data)

# train_data['ApplicantIncome']=np.log(train_data['ApplicantIncome'])

# print(np.log(train_data['ApplicantIncome']+train_data['CoapplicantIncome']),train_data['ApplicantIncome'])
y=train_data["Loan_Status"]

train_data.drop("Loan_Status",inplace=True,axis=1)

In [71]:
train_data['Gender']=train_data['Gender'].astype('category').cat.codes
train_data['Education']=train_data['Education'].astype('category').cat.codes
train_data['Self_Employed']=train_data['Self_Employed'].astype('category').cat.codes
train_data['Property_Area']=train_data['Property_Area'].astype('category').cat.codes
# train_data['Loan_Status']=testing_data['Loan_Status'].astype('category').cat.codes
# train_data['Married+Dependents']=train_data['Married+Dependents'].astype('category').cat.codes
# train_data['Dependents']=pd.get_dummies(train_data['Dependents'])


testing_data['Gender']=testing_data['Gender'].astype('category').cat.codes
testing_data['Education']=testing_data['Education'].astype('category').cat.codes
testing_data['Self_Employed']=testing_data['Self_Employed'].astype('category').cat.codes
testing_data['Property_Area']=testing_data['Property_Area'].astype('category').cat.codes
# testing_data['Loan_Status']=testing_data['Loan_Status'].astype('category').cat.codes
# testing_data['Married+Dependents']=testing_data['Married+Dependents'].astype('category').cat.codes

In [7]:
train_data['Married+Dependents'].value_counts()

pd.isnull(testing_data).sum()

Gender                0
Education             0
Self_Employed         0
LoanAmount            0
Credit_History        0
Property_Area         0
Total_Income          0
Married+Dependents    0
EMI                   0
dtype: int64

In [23]:
x_train, x_cv, y_train, y_cv = train_test_split(train_data,y, test_size =0.3)
model = LogisticRegression()
model.fit(x_train, y_train)

# pred_cv = model.predict(x_cv)
# accuracy_score(y_cv,pred_cv)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [9]:
pred_test = model.predict(testing_data)

In [72]:
i=1
kf = StratifiedKFold(n_splits=5,random_state=1,shuffle=True)

print(pd.isnull(train_data).sum())
print(pd.isnull(testing_data).sum())

for train_index,test_index in kf.split(train_data,y):
     print('\n{} of kfold {}'.format(i,kf.n_splits))
     xtr,xvl = train_data.loc[train_index],train_data.loc[test_index]
     ytr,yvl = y[train_index],y[test_index]
        
#      print(yvl.value_counts())
#      model2=RandomForestClassifier(n_estimators=100)
#      model2.fit(xtr,ytr)
#      print(accuracy_score(yvl,model2.predict(xvl)))
     model = LogisticRegression(random_state=1)
     model.fit(xtr, ytr)
     pred_test = model.predict(xvl)
     score = accuracy_score(yvl,pred_test)
     print('accuracy_score',score)
     i+=1


pred_test = model2.predict(testing_data)
pred=model.predict_proba(xvl)[:,1]

# print(pred)
# from sklearn import metrics
# fpr, tpr, _ = metrics.roc_curve(yvl,pred)
# auc = metrics.roc_auc_score(yvl, pred)
# plt.figure(figsize=(12,8))
# plt.plot(fpr,tpr,label="validation, auc="+str(auc))
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.legend(loc=4)
# plt.show()

Gender            0
Dependents        0
Education         0
Self_Employed     0
LoanAmount        0
Credit_History    0
Property_Area     0
HasDependent      0
IsSemiUrban       0
Total_Income      0
EMI               0
dtype: int64
Gender            0
Dependents        0
Education         0
Self_Employed     0
LoanAmount        0
Credit_History    0
Property_Area     0
HasDependent      0
IsSemiUrban       0
Total_Income      0
EMI               0
dtype: int64

1 of kfold 5
accuracy_score 0.8629032258064516

2 of kfold 5
accuracy_score 0.9354838709677419

3 of kfold 5
accuracy_score 0.9262295081967213

4 of kfold 5
accuracy_score 0.9180327868852459

5 of kfold 5
accuracy_score 0.9262295081967213


In [73]:
submission=pd.read_csv("Sample_Submission.csv")
submission['Loan_ID']=original_Ids
submission['Loan_Status']=pred_test
pd.DataFrame(submission, columns=['Loan_ID','Loan_Status']).to_csv('logistic4.csv')

In [74]:
pred_test

array(['Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'N', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'N',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 